## Scala:  a functional programming approach (part 1)

#### Abstract
This document is a efford to introduce the strengths and benefits of functional programming in scala.

We do not claim intelectual property of all the material presented.
We specifically refer to the original resources whenever is needed.

The presentation path of the consepts is still under consideration and may be changed in future reviews.


In [ ]:
//Based on the above

def f(x: Int) = { x + 1 }    // Is a pure function

val g = (x: Int) => s"$x" // Is a pure function

def impure1(x: Int) = { //Is impure (why?)
  println(x)
} 

def impure2(x: Double) = 10 / x  //Is umpure (why?)

def impure3(x: Int) = if (x > 0) x else sys.error("only positive numbers") //Definitely impure

We can call evaluate/execute without fear the pure functions but what about the impure ones?

In [ ]:
val f1 = f(1)
val imp1 = impure1(1)
val imp2 = impure2(0) //Do at your own risk.

How would you test function __impure1__?

It might not be so obvious here (in the scala notebook environment) but the call of __impure2(0)__ causes a __RuntimeException__.

It is more explicitly defined in method __impure3__


In [ ]:
val imp3 = impure3(-2)

#### Composability

In [ ]:
 
// h = g o f
val gof = g compose f _ //Note the use of the underscore

val `gof(1)` = gof(1)

val impure4 = impure3 _ compose f _  //Impure function composition produces more impure functions
//  impure4 (-1) throws ...

### Purity and real world

So far so good, function purity is desired in programs, but the reality is devastating. All the programs that matter to us "contain" side effects. 

__So is this introduction all in vain?__

 Purity matters even if the overall execution of our programs is impure. What we actually desire is to 
_separate_ the __impure parts__ of our programs from the __pure__ ones. That means that we write the _"business logic"_ or _"logic"_ of our program in pure functions and hand the results to impure functions to do the "dirty work".
In our context the "dirty work" is _side effects_ of all kinds.

Let's proceed in a more concrete example

#### Example: Refactoring effects.

*resource: [Functional programming in scala: Chapter 13](https://www.amazon.com/Functional-Programming-Scala-Paul Chiusano/dp/1617290653/ref=sr_1_1?ie=UTF8&qid=1504870248&sr=8-1&keywords=functional+programming+in+scala)
 
Consider the above fragment of code which calculates the winner of a contest: 

In [ ]:
//[Program 1]: decide the winner of a contest.
case class Player(name: String, score: Int)

def contest(p1: Player, p2: Player): Unit = {
  if(p1.score > p2.score)
    println(s"${p1.name} wins!")
  else if(p2.score > p1.score)
    println(s"${p2.name} wins!")
  else
    println("It is a draw!")
}

// Who is the winner?
contest(Player("fpas", 5), Player("gsmyrn", 10)) 

We are done right? The program works and prints the correct results.

This program, though simple, has some flows. One of them is that it cannot be tested (easily). 
This is because the the actual "logic" of computing the winner is interleaved with the mechanism of printing the result.

Let's change this.

In [ ]:
//[Program 2]: Split the logic from the effect.
case class Player(name: String, score: Int)

def computeWinner(p1: Player, p2: Player): Player = {
  if(p1.score > p2.score) p1
  else p2 
}

def displayWinner(p: Player): Unit = {
 println(s"${p.name} wins!")
}

//This is function composition
def contest(p1: Player, p2: Player) = displayWinner(computeWinner(p1,p2))

// Who is the winner?
contest(Player("fpas", 5), Player("gsmyrn", 10)) 

With this simple refactor the contest function is a composition of two other functions. The first part is a __pure__ function containing the logic and the second part is an __impure__ function that dispays the results.
The overall ```contest``` function is __impure__.

> __Take away 1__

> Given an impure function of type ```f: A => B``` we can split into to other functions.
> - A __pure__ function of type ``` A => D```  where ```D``` __describes__ the result of ```f```.
> - An __impure__ function of type ```D => B```, which __interprets__ ```D``` and executes the result.

That is what we strive to do in functional programming. Describe all the operations of a program in __composed pure functions__ and __promote/push/delay__ the (side) __effects execution__ to the end of the "chain". 

The carefull reader, by now, will have discover an incosistency between ```Program 1``` and ```Program 2```. The wo programs are not equivalent.

What happened to the draw case in ```Program 2```?
This was intentional and the inconsistency is caused by the fact that our pure function ```computeWinner``` cannot handle the case of a draw. 

It cannot handle it because there is no obvious result of type ```Player``` to return in the case of scores equality.

#### A closer look at side effects.

So a pure function such as ```computeWinner``` cannot handle with the cases where there is no winner.

_This is natural because a pure function must define an output for each given input_. 

But in our case we want the function to handle cases of player pairs where there is no output (winner).
That is a better implementation of the ```computeWinner``` would be:

In [ ]:
//[Program 3]: A better computeWinner

def computeWinner(p1: Player, p2: Player): Player = {
  if(p1.score > p2.score) p1
  else if (p2.score > p1.score) p2
  else null //null is a special value for this function that denotes that there is no winner.
}


Problem solved ! The function remains pure ( `null` is value of type `Player` ). Let's try it !

In [ ]:
//[Program 3 - continue]

def displayWinner(p: Player): Unit = {
 println(s"${p.name} wins!")
}

//This is function composition
def contest(p1: Player, p2: Player) = displayWinner(computeWinner(p1,p2))


// Who is the winner?
contest(Player("fpas", 10), Player("gsmyrn", 10)) 

Not what we expected a ```NullPointerException``` that is because `displayWinner` must learn to handle `null` values of players.

We can do that!

_But wait, do we want to do that?_ 

_We have done it many times and works but is there __a better way__?_

We don't want to handle null cases in our code like that. Because null leads to hidden side effects like exceptions.

> We want the end user of the pure function `computeWinner` to have a good __description__ type for its result.

So the correct question is: 

__Which data type is appropriate to describe the side effect of partial functions?__

An alternative more functional approach to the "partial function" side effect problem is the `Option[A]` type.


In [29]:
//[Program 4]: computeWinner with Option[A]
case class Player(name: String, score: Int)

def computeWinner(p1: Player, p2: Player): Option[Player] = {
  if(p1.score > p2.score) Some(p1)
  else if (p2.score > p1.score) Some(p2)
  else None
}

def displayWinner(p: Option[Player]): Unit = p match { //Pattern match instead of 'if' construct
 case Some(p) => println(s"${p.name} wins!")
 case None => println(s"It's a draw")
}

//This is function composition
def contest(p1: Player, p2: Player) = displayWinner(computeWinner(p1,p2))

// Who is the winner? (handles draw)
contest(Player("fpas", 5), Player("gsmyrn", 5)) 



fpas wins!


defined class Player
defined function computeWinner
defined function displayWinner
defined function contest

`Program 4` implementation is real close to `Program 3` implementation which used `null`  in place of `None`.

> The __key difference__ is that according to __Take away 1__ we realized that the pure function `f: A =>D` ( `computeWinner`) used a data type `D` (`Player`) that __was not expressive enough to describe the side effect of "partial function"__


So we chose an "embelised type" to __describe__ the absense of result value at some cases. 

Note, that now every one that reads the signature

```def computeWinner(p1: Player, p2: Player): Option[Player]```

knows that the method does something that may or may not return a value. The side effect is described and is not hidden as in the case of `null` usage, or any other programming convention.

### Conclusion
 - Scala is an expressive language that is able to support functional programming paradigm.

 - In functional programming paradigm we try to __describe__ with higher constructs the execution of programs.
 
 - We also try to handle side effects using contained and controlled parts of the program that __interpret__
 how to execute the side effects.
 
 - Side effects are __described__ in functional programming using more expressive types ("embelished types").
 
 - Above all functional programming tries to promote __Composability__ as a mean to achieve all the above goals.


 _Fotios Paschos, `@fpaschos` Sep, 2017_

### Structures and concepts in functional programming
_TODO_


#### Functors
List is a functor


In [30]:
// Pure functions are composable.
classpath.add("org.typelevel" %% "cats-core" % "1.0.0-MF")

Adding 0 artifact(s)


In [31]:
//Advanced-scala 3.5.1
import cats.Functor
import cats.instances.list._

val initial = List(1,2,3,4)

//Apply map directly
val res1 = Functor[List].map(initial)(_ + 1)

//Using lift
val inc = (x: Int) => x + 1
val lifted = Functor[List].lift(inc)
val res2 = lifted(initial)



import cats.Functor
import cats.instances.list._
initial: List[Int] = List(1, 2, 3, 4)
res1: List[Int] = List(2, 3, 4, 5)
inc: Int => Int = <function1>
lifted: List[Int] => List[Int] = <function1>
res2: List[Int] = List(2, 3, 4, 5)